# IFRS 9 ECL Calculation

**Objective**:
- Load my **best PD model** (e.g., tuned XGBoost) to get Probability of Default for each loan.
- Assume or derive key components:
  - **EAD** (Exposure at Default)
  - **LGD** (Loss Given Default)
- Perform a **Stage 1 ECL** calculation: 
  \[
  ECL = PD \times LGD \times EAD
  \]
- (Optional) Introduce a simple macro scenario (e.g., stressed PD = PD × 1.2) to illustrate IFRS 9 forward-looking approach.

**Context**:
In IFRS 9, banks must incorporate forward-looking information and stage the assets (Stage 1, 2, 3). For this simplified demonstration:
- I'll treat all loans as Stage 1 (one-year ECL).
- I'll not do advanced staging triggers or multi-year transitions here.
- I'll show how model-based PD flows into an ECL figure for each loan or the whole portfolio.


### Load Data & Best Model

In [5]:
import pandas as pd
import numpy as np
import joblib

# Load the cleaned dataset (the same one used for final modeling)
df_cleaned = pd.read_csv("../data/credit_risk_dataset_cleaned.csv")
print("Cleaned dataset shape:", df_cleaned.shape)

# If your best model was the tuned XGBoost, load it from a pickle or joblib if you saved it
# Or re-instantiate & re-fit quickly if needed (less ideal).
import joblib

best_xgb = joblib.load("../models/best_xgb.pkl")  # Example path if you saved the model
# If you haven't saved it, you can re-run the training code or do it right here.

df_cleaned.head()

Cleaned dataset shape: (29465, 13)


,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_to_income_ratio
0,22.0,59000.0,RENT,123.0,PERSONAL,D,35000,16.02,1,0.59,Y,3,0.593210
1,21.0,9600.0,OWN,5.0,EDUCATION,B,1000,11.14,0,0.10,N,2,0.104156
2,25.0,9600.0,MORTGAGE,1.0,MEDICAL,C,5500,12.87,1,0.57,N,3,0.572857
3,23.0,65500.0,RENT,4.0,MEDICAL,C,35000,15.23,1,0.53,N,2,0.534343
4,24.0,54400.0,RENT,8.0,MEDICAL,C,35000,14.27,1,0.55,Y,4,0.643371


### Encoding and Generating PD

The best model expects the same one-hot encoding as in modeling. I'll replicate that here, then predict the PD.

In [41]:
# Identify any object columns for one-hot encoding
cat_cols = df_cleaned.select_dtypes(include=['object']).columns
df_encoded_ifrs9 = pd.get_dummies(df_cleaned, columns=cat_cols, drop_first=True)

# Separate features (X) -- Drop `loan_status` if present.
X_all = df_encoded_ifrs9.drop(columns=['loan_status'], errors='igone')

# Predict Probability of Default using the tuned XGBoost
pd_all = best_xgb.predict_proba(X_all)[:,1]

print(f'Predicted PD array shape: {pd_all.shape}')
df_encoded_ifrs9['PD'] = pd_all
df_encoded_ifrs9.head()

Predicted PD array shape: (29465,)


,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_cred_hist_length,loan_to_income_ratio,person_home_ownership_OTHER,...,loan_intent_PERSONAL,loan_intent_VENTURE,loan_grade_B,loan_grade_C,loan_grade_D,loan_grade_E,loan_grade_F,loan_grade_G,cb_person_default_on_file_Y,PD
0,22.0,59000.0,123.0,35000,16.02,1,0.59,3,0.593210,False,...,True,False,False,False,True,False,False,False,True,0.983830
1,21.0,9600.0,5.0,1000,11.14,0,0.10,2,0.104156,False,...,False,False,True,False,False,False,False,False,False,0.028782
2,25.0,9600.0,1.0,5500,12.87,1,0.57,3,0.572857,False,...,False,False,False,True,False,False,False,False,False,0.998763
3,23.0,65500.0,4.0,35000,15.23,1,0.53,2,0.534343,False,...,False,False,False,True,False,False,False,False,False,0.995976
4,24.0,54400.0,8.0,35000,14.27,1,0.55,4,0.643371,False,...,False,False,False,True,False,False,False,False,True,0.999429


### Define LGD & EAD, Calculate ECL

**Assumptions**:

- **PD** (Probability of Default): from best XGB model.
- **EAD** (Exposure at Default): I'll use `loan_amnt` as a proxy. (one-year exposure).
- **LGD** (Loss Given Default): I'll asume a constant 0.45 (45%), typical for ensucred retails loans, though banks often have internal LGD models.

Then for each loan *i*:  
\[
ECL_i = PD_i \times LGD \times EAD_i
\]
 
Finally, I can sum or average across the portfolio for a total or average ECL.

In [45]:
LGD = 0.45

# EAD is loan_amnt from the original cleaned DataFrame
df_encoded_ifrs9['EAD'] = df_cleaned['loan_amnt']

df_encoded_ifrs9['ECL_baseline'] = (df_encoded_ifrs9['PD'] * LGD * df_encoded_ifrs9['EAD'])

avg_ecl = np.mean(df_encoded_ifrs9['ECL_baseline'])
total_ecl = np.sum(df_encoded_ifrs9['ECL_baseline'])


print(f"Average Baseline ECL: {avg_ecl:.2f}")
print(f"Total Baseline ECL: {total_ecl:.2f}")

df_encoded_ifrs9[["PD", "EAD", "ECL_baseline"]].head()

Average Baseline ECL: 1068.32
Total Baseline ECL: 31478068.67


,PD,EAD,ECL_baseline
0,0.983830,35000,15495.322943
1,0.028782,1000,12.952028
2,0.998763,5500,2471.938208
3,0.995976,35000,15686.626136
4,0.999429,35000,15741.009265


### Macro Scenario: Stressed PD

I'll mutiply the PD by 1.2 to represent a downturn scenario. This is a simplistic approach:

\[
PD_{stressed} = 1.2 \times PD_{baseline}
\]

Then recalculate ECL with the same LGD and EAD.

In [47]:
df_encoded_ifrs9["PD_stressed"] = np.minimum(df_encoded_ifrs9["PD"] * 1.2, 1.0)
# cap at 1.0, just to avoid PD > 100%

df_encoded_ifrs9["ECL_stressed"] = (df_encoded_ifrs9["PD_stressed"] * LGD * df_encoded_ifrs9["EAD"])

print(f"Average Stressed ECL: {np.mean(df_encoded_ifrs9['ECL_stressed'])}")
print(f"Total Stressed ECL: {np.sum(df_encoded_ifrs9['ECL_stressed'])}")

# Compare to baseline
print(f"ECL Increase from baseline (total): {np.sum(df_encoded_ifrs9['ECL_stressed'])} - {np.sum(df_encoded_ifrs9['ECL_baseline'])}")

Average Stressed ECL: 1137.9504970544758
Total Stressed ECL: 33529711.395710133
ECL Increase from baseline (total): 33529711.395710133 - 31478068.669319652


## Conclusion

1. **Baseline ECL**:
    - **Average**: \$1,068.32
    - **Total**: \$31,478,068.67
  
2. **Stressed ECL**:
    - **Average**: \$1,137.95
    - **Total**: \$33,529,711.39
    - **Increase** from baseline:
       \[
       33,529,711.39 - 31,478,068.67 \approx \$2.05 \text{ million}
     \]

This simple demonstration shows how **IFRS 9 forward-looking** ECL rises under worse macro conditions.

**Summary** for this portfolio project:
- I used a **tuned XGBoost** model for Probability of Default (PD).
- Calculated **Stage 1 ECL** = PD x LGD x EAD.
- Showed a **stressed** PD scenario to highlights IFRS 9's forward-looking req